In [1]:
import pandas as pd
import requests
import time
import re

import aiohttp
import asyncio

## Data Extraction
---

In [2]:
URL = "http://oscars.yipitdata.com/"

In [15]:
def get_data(url: str) -> dict:
    """
    Fetches the data from a remote URL and returns it as a pandas DataFrame.

    Returns:
        pd.DataFrame: The Yipit dataset.
    """
    response = requests.get(url)
    max_tries = 5
    wait_time = 0

    while max_tries > 0:
        try:
            # print(url)
            if response.status_code == 403:
                return {}  ## 403 Forbidden
            elif response.status_code != 200:
                raise Exception(
                    f"Failed to fetch data: {response.status_code}, trying again in {wait_time} seconds..."
                )

            data = response.json()
            # print(f"Data fetched successfully from {url} - {response.status_code}")
            return data

        except Exception as e:
            print(f"Error: {e}")
            wait_time = 0.5 * (6 - max_tries)
            time.sleep(wait_time)
            max_tries -= 1

    print(
        f"Failed to fetch data after {max_tries} attempts.  Returning empty dict."
    )
    return {}


async def get_data_async(url: str) -> dict:
    """
    Asynchronously fetches data from a remote URL and returns it as a dictionary.

    Args:
        url (str): The URL to fetch data from.

    Returns:
        dict: The data from the URL, as a dictionary.  Returns an empty dict on final failure.
    """
    max_tries = 5

    while max_tries > 0:
        try:
            async with aiohttp.ClientSession() as session:
                async with session.get(url) as response:
                    if response.status != 200:
                        raise aiohttp.ClientError(
                            f"Failed to fetch data: {response.status}"
                        )
                    elif response.status == 403:
                        return {}  ## 403 Forbidden

                    data = await response.json()
                    return data

        except aiohttp.ClientError:
            wait_time = 0.5 * (6 - max_tries)
            await asyncio.sleep(wait_time)
            max_tries -= 1
    print(
        f"Failed to fetch data after {max_tries} attempts.  Returning empty dict."
    )
    return {}

In [4]:
response_data: dict = get_data(URL)

In [5]:
films_json: pd.DataFrame = (
    pd.DataFrame(response_data["results"])
    .explode("films")
    .reset_index(drop=True)
)

films = pd.concat(
    [
        pd.json_normalize(films_json["films"]),
        films_json.drop(columns=["films"]),
    ],
    axis=1,
)

films.columns = [i.strip().replace(" ", "_").lower() for i in films.columns]

films = films[["film", "year", "wiki_url", "winner", "detail_url"]]

films.head(2)

,film,year,wiki_url,winner,detail_url
0,Wings,1927 / 28 [A] (1st),http://en.wikipedia.org/wiki/Wings_(1927_film),True,http://oscars.yipitdata.com/films/Wings_(1927_...
1,The Racket,1927 / 28 [A] (1st),http://en.wikipedia.org/wiki/The_Racket_(1928_...,False,http://oscars.yipitdata.com/films/The_Racket_(...


In [16]:
films["film_extra_details"] = films["detail_url"].apply(
    get_data
)  # 3min sync requests

In [17]:
# 1-2sec async requests -- it can only be run in .py file
# films['film_extra_details_async'] = films['Detail URL'].apply(get_data_async)

In [18]:
films_details = pd.concat(
    [
        films.drop(columns=["film_extra_details"]),
        pd.json_normalize(films["film_extra_details"]),
    ],
    axis=1,
)

films_details.columns = [
    i.strip().replace(" ", "_").lower() for i in films_details.columns
]

films_details.head(1)

,film,year,wiki_url,winner,detail_url,production_company,release_dates,running_time,budget,cinematography,...,produced_by,screenplay_by,starring,story_by,title,written_by,based_on,box_office,narrated_by,production_companies
0,Wings,1927 / 28 [A] (1st),http://en.wikipedia.org/wiki/Wings_(1927_film),True,http://oscars.yipitdata.com/films/Wings_(1927_...,Famous Players-Lasky,"August 12, 1927 ( 1927-08-12 )",Original release: 111 minutes [ 2 ] Restorati...,US$ 2 million [ 4 ],Harry Perry,...,Lucien Hubbard Adolph Zukor Jesse L. Lasky ...,Hope Loring Louis D. Lighton,"Clara Bow Charles ""Buddy"" Rogers Richard Arl...",John Monk Saunders,Wings,Titles: Julian Johnson,NaN,NaN,NaN,NaN


In [19]:
films_details[films_details["release_dates"].isnull()]

,film,year,wiki_url,winner,detail_url,production_company,release_dates,running_time,budget,cinematography,...,produced_by,screenplay_by,starring,story_by,title,written_by,based_on,box_office,narrated_by,production_companies
56,Les Misérables,1935 (8th),http://en.wikipedia.org/wiki/Les_Mis%C3%A9rabl...,False,http://oscars.yipitdata.com/films/Les_Mis%C3%A...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254,Who's Afraid of Virginia Woolf?,1966 (39th),http://en.wikipedia.org/wiki/Who%27s_Afraid_of...,False,http://oscars.yipitdata.com/films/Who%2527s_Af...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
403,Secrets & Lies,1996 (69th),http://en.wikipedia.org/wiki/Secrets_%26_Lies_...,False,http://oscars.yipitdata.com/films/Secrets_%26_...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,Les Misérables,2012 (85th),http://en.wikipedia.org/wiki/Les_Mis%C3%A9rabl...,False,http://oscars.yipitdata.com/films/Les_Mis%C3%A...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# films_details.to_csv("data/films_data.csv", index=False)

## Data Cleaning
---

In [22]:
films_details = pd.read_csv("data/raw_films_data.csv")

print(films_details.shape)
films_details.head(2)

(520, 6)


,film,year,wiki_url,winner,detail_url,budget
0,Wings,1927 / 28 [A] (1st),http://en.wikipedia.org/wiki/Wings_(1927_film),True,http://oscars.yipitdata.com/films/Wings_(1927_...,US$ 2 million [ 4 ]
1,The Racket,1927 / 28 [A] (1st),http://en.wikipedia.org/wiki/The_Racket_(1928_...,False,http://oscars.yipitdata.com/films/The_Racket_(...,NaN


In [23]:
films_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   film        520 non-null    object
 1   year        520 non-null    object
 2   wiki_url    520 non-null    object
 3   winner      520 non-null    bool  
 4   detail_url  520 non-null    object
 5   budget      422 non-null    object
dtypes: bool(1), object(5)
memory usage: 20.9+ KB


#### Cleaning budget cases

1. quanty and text -> '$25 million'
2. quantity -> $800,000
3. Diferent currencies -> 
   1. $ -> Dollar
   2. ₤ -> Euros
   3. £ -> Pounds

In [24]:
CURRENCY_CONVERSION = {"$": 1, "£": 1.32, "₤": 1.2}

MULTIPLIER_CONVERSION = {"": 1, "million": 1e6, "billion": 1e9}


def clean_budget(x):
    if pd.isna(x):
        return 0

    budget_str = str(x).strip().lower()
    pattern = r"([$₤£])+[\s]*([0-9]+(?:[.|,][0-9]+)?)[\s]*((?:[a-z]+)?)"
    search_matches = re.search(pattern, budget_str)

    if not search_matches:
        return -1

    final_budget = (
        float(search_matches.group(2).replace(",", ""))
        * CURRENCY_CONVERSION[search_matches.group(1)]
        * MULTIPLIER_CONVERSION[search_matches.group(3)]
    )

    final_budget = final_budget if final_budget < 1e7 else 1e7

    return final_budget

In [25]:
films_details["budget_clean"] = films_details["budget"].apply(clean_budget)

films_details["year_clean"] = (
    films_details["year"]
    .fillna("0")
    .apply(lambda x: re.search(r"([0-9]{4})", str(x)).group(1))
)

In [ ]:
# films_details.to_csv("data/films_data_clean.csv", index=False)